<h1>MADATORY PYTHON LIBRARIES</h1>

In [ ]:
import folium #from conda prompt window: conda install folium
from folium import plugins
from owslib.wms import WebMapService #from conda prompt window: conda install OWSLib
from IPython.core.display import HTML
folium.__version__

<H1>WEB MAP SERVICES</H1>

Web Map Services are available for almost all products within the Copernicus Marien Service.<br>
You can always check if a given product offers this service by exploring the 'services' tab in the Copernicus Marine Service Cataloge detailed view:

In [ ]:
product_id = "GLOBAL_ANALYSIS_FORECAST_PHY_001_024"
detail_view = "http://marine.copernicus.eu/services-portfolio/access-to-products/?option=com_csw&view=details&product_id="+product_id
print(detail_view)

Let's play with such service for getting a quick view of the variables of the above product

<h1>WMS - MapViews</h1>

<ul><li><h2>2D datasets</li></ul></h2>

Let's choose one of the above 2D datasets an display its variables; or example, the hourly mean dataset:

In [ ]:
dataset = 'global-analysis-forecast-phy-001-024-hourly-t-u-v-ssh'

In [ ]:
url = 'http://nrt.cmems-du.eu/thredds/wms/'+dataset
wms = WebMapService(url)

In [ ]:
for layer in wms.contents.keys():
    print(layer + ' = ' + wms[layer].title)

Let's choose one of the above 2D variables and calculate the center of its bounding box:

In [ ]:
target = 'thetao'

In [ ]:
lon = (wms[target].boundingBox[0] + wms[target].boundingBox[2]) / 2.
lat = (wms[target].boundingBox[1] + wms[target].boundingBox[3]) / 2.
center = lat, lon
print(center)

Let's now display the chosen variable on a map centered in the abobe lat,lon point:

In [ ]:
m = folium.Map(
    location=center,
    zoom_start=2,
    control_scale=True
)
style = [wms[target].styles[key]['title'] for key in wms[target].styles.keys()][0]
w = folium.raster_layers.WmsTileLayer(
    url=url,
    name=wms[target].title,
    styles=style,
    fmt='image/png',
    transparent=True,
    layers=target,
    overlay=True,
)
w.add_to(m)
m

Let's add a legend so that we know what the above color means:

In [ ]:
legend = [wms[target].styles[key]['legend'] for key in wms[target].styles.keys()][0]
print(legend)

In [ ]:
m = folium.Map(
    location=center,
    zoom_start=2,
    control_scale=True
)

style = [wms[target].styles[key]['title'] for key in wms[target].styles.keys()][0]
w = folium.raster_layers.WmsTileLayer(
    url=url,
    name=wms[target].title,
    styles=style,
    fmt='image/png',
    transparent=True,
    layers=target,
    overlay=True,
)
w.add_to(m)
legend_html = '<div style="position: fixed; bottom: 50px; right: 50px; z-index:9999"><img src="'+legend+'"></div>'
m.get_root().html.add_child(folium.Element(legend_html))
m

Let's zoom around the UK:

In [ ]:
m = folium.Map(
    location=[55.9533, 1.888],
    zoom_start=5,
    control_scale=True
)

style = [wms[target].styles[key]['title'] for key in wms[target].styles.keys()][0]
w = folium.raster_layers.WmsTileLayer(
    url=url,
    name=wms[target].title,
    styles=style,
    fmt='image/png',
    transparent=True,
    layers=target,
    overlay=True,
)
w.add_to(m)

legend_html = '<div style="position: fixed; bottom: 50px; right: 50px; z-index:9999"><img src="'+legend+'"></div>'
m.get_root().html.add_child(folium.Element(legend_html))
m

Let's reset the color scale ranges to be able to distinguish better the transitions:

In [ ]:
COLORSCALERANGE = '-2.0,16.0'

In [ ]:
legend = [wms[target].styles[key]['legend'] for key in wms[target].styles.keys()][0]+'&colorscalerange='+COLORSCALERANGE
print(legend)

In [ ]:
m = folium.Map(
    location=[55.9533, 1.888],
    zoom_start=5,
    control_scale=True
)

style = [wms[target].styles[key]['title'] for key in wms[target].styles.keys()][0]
w = folium.raster_layers.WmsTileLayer(
    url=url,
    name=wms[target].title,
    styles=style,
    fmt='image/png',
    transparent=True,
    layers=target,
    overlay=True,
    colorscalerange=COLORSCALERANGE,
)
w.add_to(m)
legend_html = '<div style="position: fixed; bottom: 50px; right: 50px; z-index:9999"><img src="'+legend+'"></div>'
m.get_root().html.add_child(folium.Element(legend_html))
m

But which time this correponds to? Let's check the time coverage of the target layer:

In [ ]:
ini = wms[target].timepositions[0].strip()
end = wms[target].timepositions[-1].strip()
time_coverage = (ini+'/'+end).split('/P')[0]
print(time_coverage)

Let's ask for certain times:

In [ ]:
m = folium.Map(
    location=[55.9533, 1.888],
    zoom_start=5,
    control_scale=True
)
style = [wms[target].styles[key]['title'] for key in wms[target].styles.keys()][0]
w = folium.raster_layers.WmsTileLayer(
    url=url,
    name='winter_2018_'+wms[target].title,
    styles=style,
    fmt='image/png',
    transparent=True,
    layers=target,
    overlay=True,
    colorscalerange=COLORSCALERANGE,
    time = '2018-02-10T12:30:00.000Z'
)
w2 = folium.raster_layers.WmsTileLayer(
    url=url,
    name='summer_2018_'+wms[target].title,
    styles=style,
    fmt='image/png',
    transparent=True,
    layers=target,
    overlay=True,
    colorscalerange=COLORSCALERANGE,
    time = '2018-08-10T12:30:00.000Z'
)
w.add_to(m)
w2.add_to(m)

folium.LayerControl().add_to(m)

legend_html = '<div style="position: fixed; bottom: 50px; right: 50px; z-index:9999"><img src="'+legend+'"></div>'
m.get_root().html.add_child(folium.Element(legend_html))

m

Let's see these snapshots next each other to better see the differences:

In [ ]:
m_dual = plugins.DualMap(location=[55.9533, 1.888], zoom_start=5, control_scale=True)
style = [wms[target].styles[key]['title'] for key in wms[target].styles.keys()][0]
w = folium.raster_layers.WmsTileLayer(
    url=url,
    name='winter_2018_'+wms[target].title,
    styles=style,
    fmt='image/png',
    transparent=True,
    layers=target,
    overlay=True,
    colorscalerange=COLORSCALERANGE,
    time = '2018-02-10T12:30:00.000Z'
)
w2 = folium.raster_layers.WmsTileLayer(
    url=url,
    name='summer_2018_'+wms[target].title,
    styles=style,
    fmt='image/png',
    transparent=True,
    layers=target,
    overlay=True,
    colorscalerange=COLORSCALERANGE,
    time = '2018-08-10T12:30:00.000Z'
)
w.add_to(m_dual.m1)
w2.add_to(m_dual.m2)

legend_html1 = '<div style="position: fixed; bottom: 50px; right:0px; z-index:9999"><img src="'+legend+'"></div>'
legend_html2 = '<div style="position: fixed; bottom: 50px; left:370px; z-index:9999"><img src="'+legend+'"></div>'
m_dual.get_root().html.add_child(folium.Element(legend_html1))
m_dual.get_root().html.add_child(folium.Element(legend_html2))

m_dual

Let's animate the target layer to see its whole evolution and not just the above time selections:

In [ ]:
w = folium.raster_layers.WmsTileLayer(
    url=url,
    name=wms[target].title+' at '+wms[target].elevations[0].strip(),
    styles=style,
    fmt='image/png',
    transparent=True,
    layers=target,
    overlay=True,
    elevation = wms[target].elevations[0].strip(),
    colorscalerange=COLORSCALERANGE
)

In [ ]:
time_layer = plugins.TimestampedWmsTileLayers(
    w,
    period='PT1H',
    time_interval='2019-03-01T00:30:00.000Z/2019-03-04T23:30:00.000Z'
)

In [ ]:
m = folium.Map(
    location=[55.9533, 1.888],
    zoom_start=5,
    control_scale=True
)
tile_layer.add_to(m)
time_layer.add_to(m)
#layer style
legend_html = '<div style="position: fixed; bottom: 50px; right:0px; z-index:9999"><img src="'+legend+'"></div>'
#add legend to map
m.get_root().html.add_child(folium.Element(legend_html))
#show map
m